In [3]:
from bs4 import BeautifulSoup, SoupStrainer
import urllib.request
from datetime import date
from dateutil.parser import parse
import pandas as pd
from tqdm import tqdm

In [41]:
def get_UFC_Event_URLs(page=1,verbose=False):
    html = urllib.request.urlopen('http://ufcstats.com/statistics/events/completed?page={}'.format(page)).read()
    if verbose:
        print('http://ufcstats.com/statistics/events/completed?page={}'.format(page))
    rows = BeautifulSoup(html,'html.parser', parse_only=SoupStrainer(class_="b-statistics__table-row"))
    urls=[]
    for row in rows:
##        print(row)
        dates = row.find_all(class_="b-statistics__date")
        if len(dates)>0:
            d=parse(dates[0].string.strip()).date()
            if d<date.today():
                a=row.find_all("a")
##                print(a)
                for url in a:
##                    print(url)
                    if url.has_attr("href"):
                        urls.append([d,url['href']])
    if verbose:
        print("Page {} completed".format(page))
    if len(urls)>0:
        nextUrls = get_UFC_Event_URLs(page=page+1,verbose=verbose)
        urls.extend(nextUrls)
    return urls
def get_Fight_results(url):
    html = urllib.request.urlopen(url).read()
    rows = BeautifulSoup(html,'html.parser',parse_only=SoupStrainer("tr"))
    fights = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols)<2: continue
        w = len(cols[0].find_all("p"))==1
        f = [p.text.strip() for p in cols[1].find_all("p")]
##        print(f[0],"beat" if w else "drew",f[1])
        fights.append([f,int(w)])
    return fights
dfs=[]
i=0
urls=get_UFC_Event_URLs()
print(len(urls))
for fight in tqdm(urls):
    d,url = fight
#     print(d)
#     if d<parse("2024-08-10").date():
#         break
    fights = get_Fight_results(url)
    for f in fights:
        a,b,w = f[0][0],f[0][1],f[1]
        name = str(d)+": "+a+" VS "+b
        dfs.append(pd.DataFrame({"Name":[name],"Fighter_1":[a],"Fighter_2":[b],"Decision":[w]},index=([i])))
        i+=1
df = pd.concat(dfs)
print(df.shape)
print(df.head(10))

721


100%|████████████████████████████████████████████████████████████████████████████████| 721/721 [06:10<00:00,  1.95it/s]


(8044, 4)
                                               Name         Fighter_1  \
0  2025-02-15: Jared Cannonier VS Gregory Rodrigues   Jared Cannonier   
1        2025-02-15: Youssef Zalal VS Calvin Kattar     Youssef Zalal   
2       2025-02-15: Edmen Shahbazyan VS Dylan Budka  Edmen Shahbazyan   
3       2025-02-15: Nazim Sadykhov VS Ismael Bonfim    Nazim Sadykhov   
4      2025-02-15: Andre Petroski VS Rodolfo Vieira    Andre Petroski   
5       2025-02-15: Jose Delgado VS Connor Matthews      Jose Delgado   
6           2025-02-15: Angela Hill VS Ketlen Souza       Angela Hill   
7       2025-02-15: Rafael Estevam VS Jesus Aguilar    Rafael Estevam   
8      2025-02-15: Gabriel Bonfim VS Khaos Williams    Gabriel Bonfim   
9         2025-02-15: Elijah Smith VS Vince Morales      Elijah Smith   

           Fighter_2  Decision  
0  Gregory Rodrigues         1  
1      Calvin Kattar         1  
2        Dylan Budka         1  
3      Ismael Bonfim         1  
4     Rodolfo Vieira 

In [45]:
def getRecord(fighter):
    dfs = []
    rows = df.query('Fighter_1=="{0}" or Fighter_2=="{0}"'.format(fighter))
    rows = rows.reset_index()
    for ind, row in rows.iterrows():
        if row['Fighter_1']==fighter:
            dfs.append(pd.DataFrame({"Name":row['Name'],"Opponent":row['Fighter_2'],"Decision":row['Decision']},index=[ind]))
        else:
            dfs.append(pd.DataFrame({"Name":row['Name'],"Opponent":row['Fighter_1'],"Decision":-row['Decision']},index=[ind]))
    return pd.concat(dfs)
print(getRecord("Brock Lesnar"))

                                           Name          Opponent  Decision
0         2016-07-09: Brock Lesnar VS Mark Hunt         Mark Hunt         0
1  2011-12-30: Alistair Overeem VS Brock Lesnar  Alistair Overeem        -1
2    2010-10-23: Cain Velasquez VS Brock Lesnar    Cain Velasquez        -1
3      2010-07-03: Brock Lesnar VS Shane Carwin      Shane Carwin         1
4         2009-07-11: Brock Lesnar VS Frank Mir         Frank Mir         1
5     2008-11-15: Brock Lesnar VS Randy Couture     Randy Couture         1
6     2008-08-09: Brock Lesnar VS Heath Herring     Heath Herring         1
7         2008-02-02: Frank Mir VS Brock Lesnar         Frank Mir        -1


In [ ]:
def knownProbabilities():
    dfs = []
    fighters = set(df['Fighter_1'].unique())
    fighters.update(set(df['Fighter_2'].unique()))
    for i in tqdm(fighters):
        rdf = getRecord(i)
        rdf = rdf.reset_index()
        p={j:[0,0] for j in fighters}
        for index, row in rdf.iterrows():
            p[row['Opponent']][0]+=(row['Decision']+1)/2
            p[row['Opponent']][1]+=1
        dfs.append(pd.DataFrame({j:p[j][0]/p[j][1] if p[j][1]>0 else None for j in fighters},index=[i]))
    return pd.concat(dfs)
kcpt = knownProbabilities()

100%|██████████████████████████████████████████████████████████████████████████████| 2568/2568 [00:59<00:00, 43.10it/s]
C:\Users\aryad\AppData\Local\Temp\ipykernel_10076\3873592330.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dfs)


In [ ]:
print(kcpt.shape)